# Comandos no Terminal:

## Baixando arquivos:

#### wget https://github.com/paaarx/semantix/raw/master/NASA_access_log_Jul95.gz
#### wget https://github.com/paaarx/semantix/raw/master/NASA_access_log_Aug95.gz

## Enviando arquivos para HDFS no Docker:

#### docker cp NASA_access_log_Aug95.gz namenode:/home
#### docker cp NASA_access_log_Jul95.gz namenode:/home

## Criando Pasta para receber arquivos:

#### hdfs dfs -mkdir /user/marcelo/data/desafio_bulk

## Enviando arquivos para pasta:

#### hdfs dfs -copyFromLocal /home/NASA_access_log_Jul95.gz /user/marcelo/data/desafio_bulk
#### hdfs dfs -copyFromLocal /home/NASA_access_log_Aug95.gz /user/marcelo/data/desafio_bulk


In [1]:
# Verificando arquivos no HDFS:
!hdfs dfs -ls /user/marcelo/data/desafio_bulk

Found 2 items
-rw-r--r--   3 root supergroup   16633316 2022-08-31 19:25 /user/marcelo/data/desafio_bulk/NASA_access_log_Aug95.gz
-rw-r--r--   3 root supergroup   20676672 2022-08-31 19:25 /user/marcelo/data/desafio_bulk/NASA_access_log_Jul95.gz


In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import *

In [3]:
spark = SparkSession.builder \
.appName('nasa') \
.master('local[*]') \
.getOrCreate()

---------------

In [4]:
# Criando DataFrame com os arquivos:

df = spark.read.text("/user/marcelo/data/desafio_bulk/*.gz")

In [5]:
# Como os Dados chegaram:

df.show(2, truncate=False)

+-------------------------------------------------------------------------------------------------+
|value                                                                                            |
+-------------------------------------------------------------------------------------------------+
|199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245           |
|unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985|
+-------------------------------------------------------------------------------------------------+
only showing top 2 rows



-------------------------------

In [6]:
# Tratando os Dados com Regexp_extract, organizando o DataFrame:

df_1 = df.select(regexp_extract('value',r'^([^\s]+\s)',1).alias('hosts'),
                            regexp_extract('value', r'((\d\d/\w{3}/\d{4}:\d{2}:\d{2}:\d{2}))', 1).alias('data'),
                            regexp_extract('value', r'^.*"\w+\s+([^\s+]+)\s+HTTP.*', 1).alias('URL'),
                            regexp_extract('value', r'^.*\s+([^\s]+)\s', 1).cast('integer').alias('codigo_HTTP'),
                            regexp_extract('value', r'^.*\s+([^\s]+)', 1).cast('integer').alias('byte'))

----------------

In [7]:
# Como Ficou o DataFrame:

df_1.show(5, truncate=False)

+---------------------+--------------------+-----------------------------------------------+-----------+----+
|hosts                |data                |URL                                            |codigo_HTTP|byte|
+---------------------+--------------------+-----------------------------------------------+-----------+----+
|199.72.81.55         |01/Jul/1995:00:00:01|/history/apollo/                               |200        |6245|
|unicomp6.unicomp.net |01/Jul/1995:00:00:06|/shuttle/countdown/                            |200        |3985|
|199.120.110.21       |01/Jul/1995:00:00:09|/shuttle/missions/sts-73/mission-sts-73.html   |200        |4085|
|burger.letters.com   |01/Jul/1995:00:00:11|/shuttle/countdown/liftoff.html                |304        |0   |
|199.120.110.21       |01/Jul/1995:00:00:11|/shuttle/missions/sts-73/sts-73-patch-small.gif|200        |4179|
+---------------------+--------------------+-----------------------------------------------+-----------+----+
only showi

-------------------------

# Perguntas do Desafio:

### 1. Número de hosts únicos ?
### 2. O total de erros 404 ?
### 3. Os 5 URLs que mais causaram erro 404 ?
### 4. Quantidade de erros 404 por dia ?
### 5. O total de bytes retornados ?

-------------

In [8]:
# Resposta 1:

df_1.select("hosts").distinct().count()

137979

----------------

In [9]:
# Resposta 2:

df_1.where(col("codigo_HTTP") == "404").count()

20901

-----------------

In [10]:
# Resposta 3:

df_1.filter('codigo_HTTP = "404"').groupby('URL').count().sort(col("count").desc()).show(5, truncate=False)

+--------------------------------------------+-----+
|URL                                         |count|
+--------------------------------------------+-----+
|/pub/winvn/readme.txt                       |2004 |
|/pub/winvn/release.txt                      |1732 |
|/shuttle/missions/STS-69/mission-STS-69.html|682  |
|/shuttle/missions/sts-68/ksc-upclose.gif    |426  |
|/history/apollo/a-001/a-001-patch-small.gif |384  |
+--------------------------------------------+-----+
only showing top 5 rows



------------

In [11]:
# Resposta 4:

df_1.filter('codigo_HTTP = "404"').groupby(df_1.data.substr(0,11).alias("data")).count().sort(desc("count")).show(5)

+-----------+-----+
|       data|count|
+-----------+-----+
|06/Jul/1995|  640|
|19/Jul/1995|  639|
|30/Aug/1995|  571|
|07/Jul/1995|  570|
|07/Aug/1995|  537|
+-----------+-----+
only showing top 5 rows



--------------

In [12]:
# Resposta 5:

df_1.select('byte').groupby().sum().show()

+-----------+
|  sum(byte)|
+-----------+
|65524314915|
+-----------+

